# 💠 Day 3 - SQL via Python: NYC School EDA

## 🛠️ SETUP

### 🔌 Import Libraries

In [1]:
import pandas as pd
import psycopg2

### 🔗 Setup Connection

In [14]:
# DB connection setup using hardcoded credentials

conn = psycopg2.connect(
    dbname='neondb',
    user='neondb_owner',
    password='npg_CeS9fJg2azZD',
    host='ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech',
    port='5432',
    sslmode='require'
)
conn.cursor()

<cursor object at 0x12ce68220; closed: 0>

### 🫣 Test query

In [3]:
query = 'SELECT * FROM nyc_Schools.high_school_directory LIMIT 5;'

df = pd.read_sql(query, conn)

df

/var/folders/c_/l3z98zw17lvfgvq2rtl96nq00000gn/T/ipykernel_43852/3624501065.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


## ✅ Day 3 Tasks

### ⭕️ How many schools are in each borough?

In [4]:
q1_query = 'SELECT borough, count(distinct dbn) FROM nyc_schools.high_school_directory GROUP BY borough'

q1_df = pd.read_sql(q1_query, conn)

/var/folders/c_/l3z98zw17lvfgvq2rtl96nq00000gn/T/ipykernel_43852/2453372343.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q1_df = pd.read_sql(q1_query, conn)


In [5]:
q1_df

,borough,count
0,Bronx,118
1,Brooklyn,121
2,Manhattan,106
3,Queens,80
4,Staten Island,10


### ⭕️ Average % ELLs per borough?

In [10]:
q2_query = '''
select 
    hsd.borough,
    avg(ell_percent)
from nyc_schools.school_demographics sd
    left join nyc_schools.high_school_directory hsd
    using(dbn)
group by borough
;
'''

In [11]:
q2_df = pd.read_sql(q2_query, conn)

q2_df

/var/folders/c_/l3z98zw17lvfgvq2rtl96nq00000gn/T/ipykernel_43852/1998390256.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q2_df = pd.read_sql(q2_query, conn)


,borough,avg
0,None,12.708867
1,Manhattan,7.572500


### ⭕️ Top 3 Special Needs Schools per Borough

In [12]:
q3_query = '''
with top_sped_rank_per_borough as
(
	select
		row_number() over (partition by hsd.borough order by sd.sped_percent desc) as rank,
		sd.dbn,
		sd.sped_percent,
		hsd.borough
	from nyc_schools.school_demographics sd 
	left join nyc_schools.high_school_directory hsd
	using(dbn)
)

select *
from top_sped_rank_per_borough
where rank <= 3
order by borough
;
'''

In [15]:
q3_df = pd.read_sql(q3_query, conn)

q3_df

/var/folders/c_/l3z98zw17lvfgvq2rtl96nq00000gn/T/ipykernel_43852/1212757813.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q3_df = pd.read_sql(q3_query, conn)


,rank,dbn,sped_percent,borough
0,1,01M450,28.8,Manhattan
1,2,01M450,27.7,Manhattan
2,3,01M450,26.7,Manhattan
3,1,01M301,35.5,None
4,2,01M332,35.3,None
5,3,01M332,32.6,None


## Conclusion

Unfortunately there seems to be an issue with the dbn columns. At least many of the dont match between High_School_Directory and School_demographics tables, which makes it hard to join both and gain insights about boroughs other than manhatten.